In [1]:
import numba as nb
import rbc as rnb

## Define a function and its signatures

In [2]:
def foo(a, b):
    return a + b

In [3]:
foo_sig = dict(
    ifoo = nb.int64(nb.int64, nb.int64),
    dfoo = nb.float64(nb.float64, nb.float64),
)

## Generate LLVM IR strings

In [4]:
foo_ir = rnb.get_llvm_ir(foo, foo_sig, target='host')

foo_ir_cuda = rnb.get_llvm_ir(foo, foo_sig, target='cuda')

In [5]:
print(foo_ir)

; ModuleID = 'foo'
source_filename = "<ipython-input-2-89a95242c6a7>"
target datalayout = "e-m:o-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-apple-darwin19.0.0"

@"_ZN08NumbaEnv8__main__7foo$241Exx" = common local_unnamed_addr global i8* null
@"_ZN08NumbaEnv8__main__7foo$242Edd" = common local_unnamed_addr global i8* null

; Function Attrs: norecurse nounwind writeonly
define i32 @"_ZN8__main__7foo$241Exx"(i64* noalias nocapture %retptr, { i8*, i32 }** noalias nocapture readnone %excinfo, i64 %arg.a, i64 %arg.b) local_unnamed_addr #0 {
entry:
  %.14 = add nsw i64 %arg.b, %arg.a
  store i64 %.14, i64* %retptr, align 8
  ret i32 0
}

; Function Attrs: norecurse nounwind readnone
define i64 @ifoo(i64 %.1, i64 %.2) local_unnamed_addr #1 {
entry:
  %.14.i = add nsw i64 %.2, %.1
  ret i64 %.14.i
}

; Function Attrs: norecurse nounwind writeonly
define i32 @"_ZN8__main__7foo$242Edd"(double* noalias nocapture %retptr, { i8*, i32 }** noalias nocapture readnone %excinfo, double %arg

In [6]:
print(foo_ir_cuda)

; ModuleID = 'foo'
source_filename = "<ipython-input-2-89a95242c6a7>"
target datalayout = "e-p:64:64:64-i1:8:8-i8:8:8-i16:16:16-i32:32:32-i64:64:64-f32:32:32-f64:64:64-v16:16:16-v32:32:32-v64:64:64-v128:128:128-n16:32:64"
target triple = "nvptx64-nvidia-cuda"

@"_ZN08NumbaEnv8__main__7foo$243Exx" = common local_unnamed_addr global i8* null
@"_ZN08NumbaEnv8__main__7foo$244Edd" = common local_unnamed_addr global i8* null

; Function Attrs: norecurse nounwind writeonly
define i32 @"_ZN8__main__7foo$243Exx"(i64* nocapture %.ret, i64 %arg.a, i64 %arg.b) local_unnamed_addr #0 {
entry:
  %.13 = add nsw i64 %arg.b, %arg.a
  store i64 %.13, i64* %.ret, align 8
  ret i32 0
}

; Function Attrs: norecurse nounwind writeonly
define i64 @ifoo(i64 %.1, i64 %.2) local_unnamed_addr #0 {
entry:
  %.4 = alloca i64, align 8
  store i64 0, i64* %.4, align 8
  %.7 = call i32 @"_ZN8__main__7foo$243Exx"(i64* nonnull %.4, i64 %.1, i64 %.2)
  %.15 = load i64, i64* %.4, align 8
  ret i64 %.15
}

; Function Attrs

## Compile LLVM IR to machine code - host

In [7]:
eng = rnb.compile(foo_ir)

## Call the functions using cfuncs

In [8]:
from ctypes import CFUNCTYPE, c_double, c_int
ifoo = CFUNCTYPE(c_int, c_int, c_int)(eng.get_function_address('ifoo'))
dfoo = CFUNCTYPE(c_double, c_double, c_double)(eng.get_function_address('dfoo'))

In [9]:
ifoo(1,2), dfoo(3.4, 2)

(3, 5.4)